In [1]:
from pymongo import MongoClient
from tqdm import tqdm
import pandas as pd
from pd_doc2vec import doc2vec



client = MongoClient()
client = MongoClient('da1.eecs.utk.edu', 27017) 
#Connects to the MongoDB, make sure youre SSH'ed into the docker

collection_list = client['twitter'].collection_names() 
#Scrapes all the collection names from the Docker
i =0
sentimentdata = []
for posts in (collection_list):
    i = i+1
    print("Processing", posts, i, "/", str(len(collection_list)))
    for post in client['twitter'][posts].find({'sentiment': {"$exists": True}}):  
        #Extracts all the entries with a sentiment field
        post.update({"Source": str(posts)}) 
        #Updates each data with the name of the collection it comes from
        sentimentdata.append(post) 

df = pd.DataFrame(sentimentdata)
# pulls data into the Pandas DataFrame

print(df.head(3))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  if sys.path[0] == '':


Processing LabeledStatuses_Sandy_K 1 / 19
Processing Users_Florence_A 2 / 19
Processing Statuses_Maria_A 3 / 19
Processing Users_Irma_A 4 / 19
Processing Statuses_Irma_C 5 / 19
Processing LabeledStatuses_Irma_A_ 6 / 19
Processing Statuses_Florence_A 7 / 19
Processing LabeledStatuses_Irma_C 8 / 19
Processing Misc_MiscFilmReviews_M 9 / 19
Processing Statuses_Florence_C 10 / 19
Processing Statuses_Irma_A 11 / 19
Processing Statuses_MiscPower_A 12 / 19
Processing LabeledStatuses_MiscTechCompanies_A 13 / 19
Processing Statuses_Maria_C 14 / 19
Processing Statuses_MiscClimateChange_A 15 / 19
Processing LabeledStatuses_MiscTechCompanies_C 16 / 19
Processing Users_Maria_A 17 / 19
Processing Statuses_Irma_K 18 / 19
Processing LabeledStatuses_Sandy_C 19 / 19
  Lat_long                   Source Topic  \
0        |  LabeledStatuses_Sandy_K   NaN   
1        |  LabeledStatuses_Sandy_K   NaN   
2        |  LabeledStatuses_Sandy_K   NaN   

                                           TweetBody TweetDat

In [2]:
print(df.columns.values) #How many Columns we have in our dataframe


['Lat_long' 'Source' 'Topic' 'TweetBody' 'TweetDate' 'TweetID' 'TweetId'
 'TweetText' 'UserID' '_id' 'classification' 'date_time' 'notes'
 'original_file' 'sentiment' 'text']


In [3]:

df['Source'].value_counts() 

#How many sources our data comes from,  different sources might label the 
#same data under different names/labels

Misc_MiscFilmReviews_M                 100000
LabeledStatuses_MiscTechCompanies_C      5114
LabeledStatuses_Sandy_K                   325
Name: Source, dtype: int64

In [4]:
print(df.loc[df['Source'] == 'Misc_MiscFilmReviews_M'].sample(4))

# This source contains the text under "text" 


      Lat_long                  Source Topic TweetBody TweetDate TweetID  \
91903      NaN  Misc_MiscFilmReviews_M   NaN       NaN       NaN     NaN   
86338      NaN  Misc_MiscFilmReviews_M   NaN       NaN       NaN     NaN   
58918      NaN  Misc_MiscFilmReviews_M   NaN       NaN       NaN     NaN   
68066      NaN  Misc_MiscFilmReviews_M   NaN       NaN       NaN     NaN   

      TweetId TweetText UserID                       _id classification  \
91903     NaN       NaN    NaN  5beee7988d75df32fe7cd40a            NaN   
86338     NaN       NaN    NaN  5beee7988d75df32fe7cbe4d            NaN   
58918     NaN       NaN    NaN  5beee7988d75df32fe7c5331            NaN   
68066     NaN       NaN    NaN  5beee7988d75df32fe7c76ed            NaN   

      date_time notes original_file sentiment  \
91903       NaN   NaN           NaN  negative   
86338       NaN   NaN           NaN  negative   
58918       NaN   NaN           NaN  positive   
68066       NaN   NaN           NaN  positive  

In [5]:
print(df.loc[df['Source'] == 'LabeledStatuses_Sandy_K'].sample(4))

# This source contains the text under "TweetBody" and has a "classification"
# field

    Lat_long                   Source Topic  \
130        |  LabeledStatuses_Sandy_K   NaN   
281        |  LabeledStatuses_Sandy_K   NaN   
81         |  LabeledStatuses_Sandy_K   NaN   
27         |  LabeledStatuses_Sandy_K   NaN   

                                             TweetBody TweetDate  \
130  Hope everyone in america stays safe in the sto...       NaN   
281  RT @SpittyP: To any and all who were affected ...       NaN   
81   @_ashhmoney I hate you. Sandy gon blow my litt...       NaN   
27   RT @Tofaial: Hurricane Sandy?  Not even intimi...       NaN   

                   TweetID TweetId TweetText       UserID  \
130  |"263017227308105728"     NaN       NaN        klw93   
281  |"265082031568982018"     NaN       NaN       kcluva   
81   |"262921713644367877"     NaN       NaN  __half_pint   
27   |"262622755701260288"     NaN       NaN  CuDi_Zoning   

                          _id  classification         date_time notes  \
130  5c0e7b47d2512c61248b1547  conversationa

In [6]:
print(df.loc[df['Source'] == 'LabeledStatuses_MiscTechCompanies_C'].sample(4))


       Lat_long                               Source      Topic TweetBody  \
105246      NaN  LabeledStatuses_MiscTechCompanies_C    twitter       NaN   
103396      NaN  LabeledStatuses_MiscTechCompanies_C  microsoft       NaN   
104773      NaN  LabeledStatuses_MiscTechCompanies_C    twitter       NaN   
100928      NaN  LabeledStatuses_MiscTechCompanies_C      apple       NaN   

                             TweetDate TweetID             TweetId  \
105246  Thu Oct 20 03:41:03 +0000 2011     NaN  126865487584968704   
103396  Wed Oct 19 13:15:07 +0000 2011     NaN  126647567877541889   
104773  Thu Oct 20 03:27:19 +0000 2011     NaN  126862030979334144   
100928  Tue Oct 18 03:17:50 +0000 2011     NaN  126134865887363072   

                                                TweetText UserID  \
105246                                      xau #twitter.    NaN   
103396  #Microsoft shows 'touch screen' for any surfac...    NaN   
104773  RT @AmySimendinger: @rachiecrewz dude you can'...  

In [7]:

df["main"] = df["sentiment"]
# we are going to build a main label that is just the sentiment

df["sentiment"] = df["Source"] + "_" + df["sentiment"]
# We add source informationg with the sentiment

df['text'].fillna(df['TweetBody'], inplace=True)
df['text'].fillna(df['TweetText'], inplace=True)
# we move the text from tweetbody into text

df['classification'] = df['classification'].fillna("unclassified")
df['Topic'] = df['Topic'].fillna("no_topic")
#we populate empty classification  fields with "0"


print(df.isna().sum())




Lat_long          105114
Source                 0
Topic                  0
TweetBody         105114
TweetDate         100327
TweetID           105114
TweetId           100327
TweetText         100327
UserID            105114
_id                    0
classification         0
date_time         105114
notes             105114
original_file     100000
sentiment              2
text                   2
main                   2
dtype: int64


In [8]:
# We need text as our X and our Y will be main, sentiment, Source, Topic, classification
#But as seen above, we have NANs in the main, text, sentiment columns! so we should drop them 
df = df.drop(['Lat_long', 'TweetBody', 'TweetDate', 'TweetID', 'TweetId', 'TweetText', 'UserID', 'date_time', 'notes', 'original_file'], axis=1)
df1 = df[df.isnull().any(axis=1)]
print(df1)

                                     Source  \
101705  LabeledStatuses_MiscTechCompanies_C   
104603  LabeledStatuses_MiscTechCompanies_C   

                                                    Topic  \
101705               #google #Nexus http://t.co/CSAeFRHc"   
104603   add yourself for more #twitter followers! net...   

                             _id classification sentiment  text  main  
101705  5c0e75e4d2512c60ce526c66   unclassified       NaN  None  None  
104603  5c0e75e4d2512c60ce5277b8   unclassified       NaN  None  None  


In [9]:
#Those dont have any usefull information, we can drop them! 
df = df.dropna()
print(df.isna().sum())
print(df.head(3))

Source            0
Topic             0
_id               0
classification    0
sentiment         0
text              0
main              0
dtype: int64
                    Source     Topic                       _id  \
0  LabeledStatuses_Sandy_K  no_topic  5c0e7b47d2512c61248b14c5   
1  LabeledStatuses_Sandy_K  no_topic  5c0e7b47d2512c61248b14c6   
2  LabeledStatuses_Sandy_K  no_topic  5c0e7b47d2512c61248b14c7   

   classification                           sentiment  \
0  conversational    LabeledStatuses_Sandy_K_positive   
1  conversational    LabeledStatuses_Sandy_K_positive   
2      irrelevant  LabeledStatuses_Sandy_K_irrelevant   

                                                text        main  
0  -1 RT @_dreamvillle: On a scale from 1-5 how s...    positive  
1  @jenfromTHEbloc Hi! I have you and all your NY...    positive  
2  #Pakistan Salesforce Social Marketing Cloud Bu...  irrelevant  


In [10]:
df = df.applymap(str)
#To make sure we dont have wierd data types lingering

In [11]:
# These are the different classifications we can do cosine similarity and check distance on

print(df.Topic.unique())
print(df.Source.unique())
print(df.classification.unique())
print(df.sentiment.unique())
print(df.main.unique())


['no_topic' 'apple' 'google' 'microsoft' 'twitter']
['LabeledStatuses_Sandy_K' 'Misc_MiscFilmReviews_M'
 'LabeledStatuses_MiscTechCompanies_C']
['conversational' 'irrelevant' 'informational' 'unclassified']
['LabeledStatuses_Sandy_K_positive' 'LabeledStatuses_Sandy_K_irrelevant'
 'LabeledStatuses_Sandy_K_negative' 'LabeledStatuses_Sandy_K_neutral'
 'Misc_MiscFilmReviews_M_positive' 'Misc_MiscFilmReviews_M_negative'
 'LabeledStatuses_MiscTechCompanies_C_positive'
 'LabeledStatuses_MiscTechCompanies_C_negative'
 'LabeledStatuses_MiscTechCompanies_C_neutral'
 'LabeledStatuses_MiscTechCompanies_C_irrelevant']
['positive' 'irrelevant' 'negative' 'neutral']


In [12]:
df=df.replace('irrelevant', 'neutral') #Combining these will make the accuracy better
print(df.main.unique())


['positive' 'neutral' 'negative']


In [13]:

print("Setting vairables")

# We pass the class 3 fields:
# 1. The DataFrame
# 2. The X value, the Text (Pandas Series)
# 3. The Y values, the labels that correspond to the text (Pandas Series)
#                   It can be a list of the names of the columns or one as a string

x = doc2vec(df, "text", ["main",  "Source", 'sentiment', 'classification', 'Topic'])

print("Scoring Model")

#returns scores of each labels accuracy from the first column passed into 
# the Y or 3 arguments, in this example it was "main"
# this uses sklearn.metrics.f1_score, you can pass in 
x.score(verbose=True)

print("Predicting on text")

# You can predict text as follows
print(x.predict_text("I dislike old cabin cruisers."))


Setting vairables
Scoring Model
splitting train and test
labeling sentences
training model


  0%|          | 0/5272 [00:00<?, ?it/s]

making predictions


/usr/local/lib/python3.5/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
100%|██████████| 5272/5272 [20:31<00:00,  4.68it/s]
/home/nwest13/Curent/Manny/pd_doc2vec.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  test['results'] = self.predict(test[X])


Scoring results
Label Score: 
[0.83327568 0.63265306 0.79946285]
Accuracy Score: 
0.8133535660091047
Predicting on text
['negative', 'Misc_MiscFilmReviews_M', 'Misc_MiscFilmReviews_M_negative', 'unclassified', 'no_topic']


In [14]:
media = pd.read_csv("Media_Tweets_1000.csv") #Tweets we want to clasify

from tqdm import tqdm
tqdm.pandas()  #To do progress_apply, it is pandas .apply but with a progess bar 


#lets label the media tweets with what the closest doc2vec label is
media["doc2vec_results"] = x.predict(media["Tweet"])



#now lets add the vector and label distances to each row so we can use randomforrest
media["results"] =  media["Tweet"].progress_apply(x.predict_sims)


sims_dict = {'apple':[], 'microsoft':[], 'LabeledStatuses_MiscTechCompanies_C_neutral':[], 'LabeledStatuses_MiscTechCompanies_C_irrelevant':[], 'LabeledStatuses_MiscTechCompanies_C_positive':[], 'google':[], 'LabeledStatuses_MiscTechCompanies_C_negative':[], 'twitter':[], 'LabeledStatuses_MiscTechCompanies_C':[], 'neutral':[], 'unclassified':[], 'no_topic':[], 'Misc_MiscFilmReviews_M':[], 'positive':[], 'negative':[], 'Misc_MiscFilmReviews_M_positive':[],'Misc_MiscFilmReviews_M_negative':[],'LabeledStatuses_Sandy_K_positive':[], 'conversational':[], 'LabeledStatuses_Sandy_K':[], 'LabeledStatuses_Sandy_K_irrelevant':[], 'informational':[], 'LabeledStatuses_Sandy_K_neutral':[], 'LabeledStatuses_Sandy_K_negative':[]}



for index, row in media.iterrows():
    results_list =row["results"]

    for i in [i[0] for i in results_list ]:
        sims_dict[i].append([rec for rec in results_list if rec[0] == i][0][1])

#This will create a dataframe from the dict and then it will be concated to the original
#This will add label distances
media_df = pd.DataFrame.from_dict(sims_dict)
media = pd.concat([media, media_df], axis=1)


#This will add the raw vector data to the row, we will need to expand it
#to be able to use it with random forrest
media["vector"] =  media["Tweet"].progress_apply(x.get_vector)
media_df = pd.DataFrame(media["vector"].values.tolist())
media = pd.concat([media, media_df], axis=1)



print(media.head(10))


  0%|          | 0/951 [00:00<?, ?it/s]/usr/local/lib/python3.5/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
100%|██████████| 951/951 [00:00<00:00, 2631.63it/s]


   Order     ID       Date                                              Tweet  \
0      1   3020  10-Sep-17  @wsvn right below is the parking lot, across f...   
1      3  67518  10-Sep-17     @WPTV #HurricanIrma pic.twitter.com/cpStsW3MXx   
2      5  20428  10-Sep-17  . @DukeEnergy says 447 people without power in...   
3      7  39812  13-Sep-17  So if your regular pickup is on a Tuesday, do ...   
4      9  60263  09-Sep-17  4 more shelters opened in Sarasota Co. bringin...   
5     10  40013  10-Sep-17  Surprisingly, still got it on Wacahoota Road i...   
6     11  36698  10-Sep-17  . @PolkCoSheriff Grady Judd says his deputies ...   
7     12  61411  14-Sep-17  @FLGovScott @SenRubioPress has anyone come to ...   
8     13  34114  08-Sep-17  Good morning West Palm Beach. Heading to @WPTV...   
9     14   2018  04-Sep-17  Gov. Rick Scott urges Floridians to plan for H...   

     manual doc2vec_results  \
0   neutral        negative   
1   neutral        positive   
2   neutral    

In [15]:
print(media.columns.values) #How many Columns we have in our dataframe


['Order' 'ID' 'Date' 'Tweet' 'manual' 'doc2vec_results' 'results'
 'LabeledStatuses_MiscTechCompanies_C'
 'LabeledStatuses_MiscTechCompanies_C_irrelevant'
 'LabeledStatuses_MiscTechCompanies_C_negative'
 'LabeledStatuses_MiscTechCompanies_C_neutral'
 'LabeledStatuses_MiscTechCompanies_C_positive' 'LabeledStatuses_Sandy_K'
 'LabeledStatuses_Sandy_K_irrelevant' 'LabeledStatuses_Sandy_K_negative'
 'LabeledStatuses_Sandy_K_neutral' 'LabeledStatuses_Sandy_K_positive'
 'Misc_MiscFilmReviews_M' 'Misc_MiscFilmReviews_M_negative'
 'Misc_MiscFilmReviews_M_positive' 'apple' 'conversational' 'google'
 'informational' 'microsoft' 'negative' 'neutral' 'no_topic' 'positive'
 'twitter' 'unclassified' 'vector' 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15
 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39
 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63
 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87
 88 89 90 91 92 93 94 95 96 97 98 99 100

In [16]:
media_cleaned = media.drop(['Order', 'ID', 'Date', 'Tweet', 'doc2vec_results', 'doc2vec_results', 'results', 'vector'], axis=1)
train_media_cleaned = media_cleaned.dropna()
train_mediax = train_media_cleaned.drop(['manual'], axis=1)
train_mediay =  train_media_cleaned['manual']

#Preparing our train set
print(train_mediax.shape)
print(train_mediay.shape)



(109, 324)
(109,)


In [17]:
print(train_mediax.columns.values)


['LabeledStatuses_MiscTechCompanies_C'
 'LabeledStatuses_MiscTechCompanies_C_irrelevant'
 'LabeledStatuses_MiscTechCompanies_C_negative'
 'LabeledStatuses_MiscTechCompanies_C_neutral'
 'LabeledStatuses_MiscTechCompanies_C_positive' 'LabeledStatuses_Sandy_K'
 'LabeledStatuses_Sandy_K_irrelevant' 'LabeledStatuses_Sandy_K_negative'
 'LabeledStatuses_Sandy_K_neutral' 'LabeledStatuses_Sandy_K_positive'
 'Misc_MiscFilmReviews_M' 'Misc_MiscFilmReviews_M_negative'
 'Misc_MiscFilmReviews_M_positive' 'apple' 'conversational' 'google'
 'informational' 'microsoft' 'negative' 'neutral' 'no_topic' 'positive'
 'twitter' 'unclassified' 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18
 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42
 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66
 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90
 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110
 111 112 113 114 115 116 117 118 1

In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

clf = RandomForestClassifier()
clf.fit(train_mediax, train_mediay)
media["rf_results"] = clf.predict(media_cleaned.drop(['manual'], axis=1))
print(media.head(10))

   Order     ID       Date                                              Tweet  \
0      1   3020  10-Sep-17  @wsvn right below is the parking lot, across f...   
1      3  67518  10-Sep-17     @WPTV #HurricanIrma pic.twitter.com/cpStsW3MXx   
2      5  20428  10-Sep-17  . @DukeEnergy says 447 people without power in...   
3      7  39812  13-Sep-17  So if your regular pickup is on a Tuesday, do ...   
4      9  60263  09-Sep-17  4 more shelters opened in Sarasota Co. bringin...   
5     10  40013  10-Sep-17  Surprisingly, still got it on Wacahoota Road i...   
6     11  36698  10-Sep-17  . @PolkCoSheriff Grady Judd says his deputies ...   
7     12  61411  14-Sep-17  @FLGovScott @SenRubioPress has anyone come to ...   
8     13  34114  08-Sep-17  Good morning West Palm Beach. Heading to @WPTV...   
9     14   2018  04-Sep-17  Gov. Rick Scott urges Floridians to plan for H...   

     manual doc2vec_results  \
0   neutral        negative   
1   neutral        positive   
2   neutral    

/usr/local/lib/python3.5/dist-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [19]:
print(media.columns.values)



['Order' 'ID' 'Date' 'Tweet' 'manual' 'doc2vec_results' 'results'
 'LabeledStatuses_MiscTechCompanies_C'
 'LabeledStatuses_MiscTechCompanies_C_irrelevant'
 'LabeledStatuses_MiscTechCompanies_C_negative'
 'LabeledStatuses_MiscTechCompanies_C_neutral'
 'LabeledStatuses_MiscTechCompanies_C_positive' 'LabeledStatuses_Sandy_K'
 'LabeledStatuses_Sandy_K_irrelevant' 'LabeledStatuses_Sandy_K_negative'
 'LabeledStatuses_Sandy_K_neutral' 'LabeledStatuses_Sandy_K_positive'
 'Misc_MiscFilmReviews_M' 'Misc_MiscFilmReviews_M_negative'
 'Misc_MiscFilmReviews_M_positive' 'apple' 'conversational' 'google'
 'informational' 'microsoft' 'negative' 'neutral' 'no_topic' 'positive'
 'twitter' 'unclassified' 'vector' 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15
 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39
 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63
 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87
 88 89 90 91 92 93 94 95 96 97 98 99 100

In [20]:
media_final = media[['Order', 'ID', 'Date', 'Tweet', 'manual', 'doc2vec_results', 'rf_results']]

In [21]:
media_score = media.dropna()
from sklearn.metrics import f1_score,  confusion_matrix
from sklearn.metrics import accuracy_score

print("Doc2Vec")
print("Label Score: ")
print(f1_score(media_score['manual'], media_score['doc2vec_results'], average=None))  # Uses train test split to get score
print("Accuracy Score: ")
print(accuracy_score(media_score['manual'], media_score['doc2vec_results']))  
print('Confusion matrix:')
print(confusion_matrix(media_score['manual'], media_score['doc2vec_results']))  

print("====-=++=+++++=======+==+==========+++++==+++===++++===+++==+++===++")
print("RandomForrest")
print("Label Score: ")
print(f1_score(media_score['manual'], media_score['rf_results'], average=None))  # Uses train test split to get score
print("Accuracy Score: ")
print(accuracy_score(media_score['manual'], media_score['rf_results']))  
print('Confusion matrix:')
print(confusion_matrix(media_score['manual'], media_score['rf_results']))  

Doc2Vec
Label Score: 
[0.54814815 0.09090909 0.15384615]
Accuracy Score: 
0.3853211009174312
Confusion matrix:
[[37  1  4]
 [33  2  6]
 [23  0  3]]
====-=++=+++++=======+==+==========+++++==+++===++++===+++==+++===++
RandomForrest
Label Score: 
[1. 1. 1.]
Accuracy Score: 
1.0
Confusion matrix:
[[42  0  0]
 [ 0 41  0]
 [ 0  0 26]]


In [22]:
media_final.to_csv("Media_Tweets_1000_bothresults.csv", index=False)

In [23]:
media.to_csv("Media_Tweets_1000_raw.csv", index=False)

In [24]:
print(x.model.most_similar('happy', topn=10))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.5/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('behalf', 0.2262553870677948), ('inferiority', 0.22282403707504272), ('braedonrice', 0.22031021118164062), ('assorted', 0.22029951214790344), ('diabolik', 0.21915343403816223), ('kirkwood', 0.2182234674692154), ('befriending', 0.21771174669265747), ('raining', 0.21722975373268127), ('reduce', 0.21531358361244202), ('rollers', 0.21437782049179077)]


In [19]:
# how to save model from https://radimrehurek.com/gensim/models/doc2vec.html
from gensim.models.doc2vec import Doc2Vec
from gensim.test.utils import get_tmpfile

fname = get_tmpfile("/home/nwest13/Curent/Manny/my_doc2vec_model")
x.model.save(fname)
x.model = Doc2Vec.load(fname)  # you can continue training with the loaded model!

# If you’re finished training a model (=no more updates, only querying, reduce memory usage), you can do:

# x.model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)



